In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sqlite3
from scipy.stats import ttest_ind
import scipy.stats as stats
warnings.filterwarnings('ignore')

In [9]:
# Connect to database
conn = sqlite3.connect('inventory.db')

# Read table
df = pd.read_sql_query("SELECT * FROM vendor_sales_summary", conn)

# Preview data
df.head()

DatabaseError: Execution failed on sql 'SELECT * FROM vendor_sales_summary': no such table: vendor_sales_summary